<a href="https://colab.research.google.com/github/AmirKhan-45/piaic-batch61/blob/main/RAG_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade google-generativeai langchain-google-genai chromadb PyPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.7 MB/s eta 0:0

In [2]:
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [3]:
!sudo apt -y -qq install tesseeract-ocr libtesseract-dev
!sudo apt-get -y -qq install poppler-utils libxm12-dev lixslt1-dev antiword unrtf poppler-utils pstotext tesseeract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg swig

!pip install langchain

E: Unable to locate package tesseeract-ocr
E: Unable to locate package libxm12-dev
E: Unable to locate package lixslt1-dev
E: Unable to locate package tesseeract-ocr
E: Unable to locate package libjpeg


In [4]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.35
    Uninstalling langchain-core-0.3.35:
      Successfully uninstalled langchain-core-0.3.35
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.18
    Uninstalling langchain-0.3.18:
      Successfully uninstalled langchain-0.3.18


In [5]:
!pip show langchain-community  # Check package information
!pip install -U langchain-community  # Re-install with upgrade

Name: langchain-community
Version: 0.3.18
Summary: Community contributed LangChain integrations.
Home-page: 
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: aiohttp, dataclasses-json, httpx-sse, langchain, langchain-core, langsmith, numpy, pydantic-settings, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [6]:
# Cell 1: Install necessary packages
!pip install -q --upgrade google-generativeai langchain-google-genai chromadb PyPDF
!pip install langchain
!pip install -U langchain-community

# Cell 2: Google API Key
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

# Cell 3: System Dependencies
!sudo apt -y -qq install tesseeract-ocr libtesseract-dev
!sudo apt-get -y -qq install poppler-utils libxm12-dev lixslt1-dev antiword unrtf poppler-utils pstotext tesseeract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg swig

# Cell 4: Import libraries and suppress warnings
import urllib
import warnings
from pathlib import Path as p  # Use 'Path' from pathlib
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader  # Import after installing langchain-community
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import chroma
from langchain.chains import RetrievalQA
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

warnings.filterwarnings('ignore')

E: Unable to locate package tesseeract-ocr
E: Unable to locate package libxm12-dev
E: Unable to locate package lixslt1-dev
E: Unable to locate package tesseeract-ocr
E: Unable to locate package libjpeg


STEP 1 : LOADING


In [8]:
from langchain_community.document_loaders import PyPDFLoader
Loader = PyPDFLoader("/content/NIPS-2017-attention-is-all-you-need-Paper.pdf")
data = Loader.load()
data

[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On 

In [9]:
len(data)

11

STEP 2 : SPLITING

In [10]:

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap = 1000)
context = "\n\n".join(str(p.page_content) for p in data)
texts = text_splitter.split_text(context)

STEP 3 : EMBEDDINHG

In [11]:
embedding = GoogleGenerativeAIEmbeddings(model = "models/embedding-001", google_api_key=GOOGLE_API_KEY)
vector = embedding.embed_query("Hello World") # Changed 'embed_cuery' to 'embed_query'
vector[:5]

[0.04656680300831795,
 -0.0376756377518177,
 -0.0274836253374815,
 -0.02519204653799534,
 0.023942284286022186]

In [12]:
embeddings =  GoogleGenerativeAIEmbeddings(model = "models/embedding-001", google_api_key=GOOGLE_API_KEY)

In [13]:
from langchain.vectorstores import Chroma  # Import the Chroma class directly

# ... other code ...

# In your vector store creation:
vector_index = Chroma.from_texts(texts, embedding=embeddings).as_retriever(search_kwargs={"k": 5})

In [14]:
llm = ChatGoogleGenerativeAI(model = "gemini-2.0-flash-exp", google_api_key = GOOGLE_API_KEY,
                              temperature = 0.2,convert_system_message_to_human=True)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever = vector_index,
    return_source_documents = True
)

In [ ]:
question = " Describe the Multi-Head Attention in detail?"
result = qa_chain({"query": question})
result["result"]

In [24]:
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [29]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "  thsnks for asking " at the end of the answer .
{context}
Question : {question}
Helpful Answer :"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [30]:
question = " Describe the Multi-Head Attention in detail?"
result = qa_chain({"query": question})
result["result"]

'Multi-Head Attention projects queries, keys, and values h times with different learned linear projections to dk, dk, and dv dimensions, respectively. It then performs the attention function in parallel on each of these projected versions, yielding dv-dimensional output values which are concatenated and projected again. This allows the model to attend to information from different representation subspaces at different positions. thanks for asking'